In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys

import random

year='2021'

In [ ]:
#Read CSV file containing the people from the West Midlands of England (output from SPENSER_notebook_6)
df_persons_NE_inactive_5categories_export_20220304_correct_dir = '/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only' # use your path
df_persons_NE_inactive_5categories_export_20220304_correct_dir_file = os.path.join(df_persons_NE_inactive_5categories_export_20220304_correct_dir, "df_persons_NE_inactive_5categories_export_correct.csv")
df_persons_NE_inactive_5categories = pd.read_csv(df_persons_NE_inactive_5categories_export_20220304_correct_dir_file, index_col=None, header=0)

## Income for inactive people will be assigned based on statistical data

## Retired:

In [ ]:
# Select those inactive retired people:
df_retired = df_persons_NE_inactive_5categories.loc[df_persons_NE_inactive_5categories['Occupation'] == 'Retired']

Marital_status_list = ['Married', 'Couple']

# Those couples:
df_retired_couple = df_retired.loc[df_retired['Marital_status'].isin(Marital_status_list)]

# Those retired single male:
df_retired_single_male = df_retired.loc[(df_retired['Marital_status'] == 'Single') & (df_retired['Sex'] == 1)]

#Those retired single female:
df_retired_single_female = df_retired.loc[(df_retired['Marital_status'] == 'Single') & (df_retired['Sex'] == 2)]


#Income values to be assigned (income in £ per week 2020/21 prices):
##Pensioners’ Incomes Series: financial year 2020 to 2021
##https://www.gov.uk/government/statistics/pensioners-incomes-series-financial-year-2020-to-2021
annual_gross_couple_2020_2021 = int(713 * 52)
annual_gross_single_male_2020_2021 = int(433 * 52)
annual_gross_single_female_2020_2021 = int(360 * 52)


df_retired_couple['Income'] = np.random.normal(int(annual_gross_couple_2020_2021/2),int(annual_gross_couple_2020_2021/2)*0.1, df_retired_couple.shape[0])
df_retired_single_male['Income'] = np.random.normal(int(annual_gross_single_male_2020_2021),int(annual_gross_single_male_2020_2021)*0.1, df_retired_single_male.shape[0])
df_retired_single_female['Income'] = np.random.normal(int(annual_gross_single_female_2020_2021),int(annual_gross_single_female_2020_2021)*0.1, df_retired_single_female.shape[0])

df_retired__updated_all = (pd.concat([df_retired_couple, df_retired_single_male, df_retired_single_female]))


In [ ]:
df_retired__updated_all['Income'].min()

In [ ]:
df_retired__updated_all['Income'].max()

In [ ]:
df_retired__updated_all.hist(column='Income', bins=50);

## Long term sick

In [ ]:
# PIP
# You can get Personal Independence Payment (PIP) if all of the following apply to you:
## you’re 16 or over
## you have a long-term physical or mental health condition or disability
## you have difficulty doing certain everyday tasks or getting around
## you expect the difficulties to last for at least 12 months from when they started
# https://www.gov.uk/pip
pip_daily_living_part_income = [61.85, 92.40] #(£ weeklyly standard allowance)
pip_mobility_part_income = [24.45, 64.50]  #(£ weekly standard allowance)


# Universal Credit
##You may be able to get Universal Credit if you’re on a low income or need help with your living costs. You could be working (including self-employed or part time) or be out of work.
#To claim you must:
##live in the UK
##be aged 18 or over (there are some exceptions if you’re 16 to 17)
##be under State Pension age
##have £16,000 or less in money, savings and investments
## https://www.gov.uk/universal-credit

universal_credit_standard_single_under25 = 265.31 #(£ extra Monthly standard allowance)
universal_credit_standard_single_25plus = 334.91 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_under25 = 416.45 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_25plus = 525.72 #(£ extra Monthly standard allowance)

universal_credit_children_first = 290.00 #(£ extra Monthly)
universal_credit_children_second = 244.58 #(£ extra Monthly)

universal_credit_extra_disability_or_health_condition = 354.28 #(£ extra Monthly)



# Select people sick
df_sick = df_persons_NE_inactive_5categories.loc[df_persons_NE_inactive_5categories['Occupation'] == 'Long term sick']

df_sick['Income'] = 0


# Calculate the income for each sick person, based on their characteristics:
for idx_person, person in df_sick.iterrows():
    
    # If person is younger than 25:
    if person['Age'] < 25 :
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                    
            # if the person (single and below 25 years) does not have any children:       
            else:
                pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                
                universal_credit_income = universal_credit_standard_single_under25 + universal_credit_extra_disability_or_health_condition
                
                income_total = 52 * pip_income + 12 *universal_credit_income
                
                df_sick.at[idx_person,'Income'] = int(income_total)
            
        
        # If person is below 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)      
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                    
            
            # if the person (married or couple and below 25 years) does not have any children:       
            else:
                pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_extra_disability_or_health_condition
                
                income_total = 52 * pip_income + 12 *universal_credit_income
                
                df_sick.at[idx_person,'Income'] = int(income_total)
            
     
    # If person is older or equal to 25:       
    else:
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
        
            # if the person (single and > 25 years) does not have any children:       
            else:
                
                pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                universal_credit_income = universal_credit_standard_single_25plus + universal_credit_extra_disability_or_health_condition
                
                income_total = 52 * pip_income + 12 *universal_credit_income
                
                df_sick.at[idx_person,'Income'] = int(income_total)
        
        # If person is above 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_disability_or_health_condition
                
                    income_total = 52 * pip_income + 12 *universal_credit_income
                
                    df_sick.at[idx_person,'Income'] = int(income_total)
            
            
            # if the person (couple/married and > 25 years) does not have any children:       
            else:
                
                pip_income =  random.randrange(int(pip_daily_living_part_income[0]),int(pip_daily_living_part_income[1]), 1) + random.randrange(int(pip_mobility_part_income[0]),int(pip_mobility_part_income[1]), 1)
                universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_extra_disability_or_health_condition
                
                income_total = 52 * pip_income + 12 *universal_credit_income
                
                df_sick.at[idx_person,'Income'] = int(income_total)


In [ ]:
df_sick['Income'].min()

In [ ]:
df_sick['Income'].max()

In [ ]:
df_sick.hist(column='Income', bins=50);

# Looking after family / home

In [ ]:
# Carers allowance
##You could get £69.70 a week if you care for someone at least 35 hours a week and they get certain benefits.
##You do not have to be related to, or live with, the person you care for.
##You do not get paid extra if you care for more than one person.
##If someone else also cares for the same person as you, only one of you can claim Carer’s Allowance.
## https://www.gov.uk/carers-allowance

carer_allowance = 69.70 #(£ weeklyly standard allowance)

# Universal Credit
##You may be able to get Universal Credit if you’re on a low income or need help with your living costs. You could be working (including self-employed or part time) or be out of work.
#To claim you must:
##live in the UK
##be aged 18 or over (there are some exceptions if you’re 16 to 17)
##be under State Pension age
##have £16,000 or less in money, savings and investments
## https://www.gov.uk/universal-credit

universal_credit_standard_single_under25 = 265.31 #(£ extra Monthly standard allowance)
universal_credit_standard_single_25plus = 334.91 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_under25 = 416.45 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_25plus = 525.72 #(£ extra Monthly standard allowance)

universal_credit_children_first = 290.0 #(£ extra Monthly)
universal_credit_children_second = 244.58 #(£ extra Monthly)

universal_credit_extra_care = 168.81 #(£ extra Monthly)


df_looking_after = df_persons_NE_inactive_5categories.loc[df_persons_NE_inactive_5categories['Occupation'] == 'Looking after home family']

df_looking_after['Income'] = 0


for idx_person, person in df_looking_after.iterrows():
    
    # If person is younger than 25:
    if person['Age'] < 25 :
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 *universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                    
            # if the person (single and below 25 years) does not have any children:       
            else:
                
                universal_credit_income = universal_credit_standard_single_under25 + universal_credit_extra_care
                
                income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                df_looking_after.at[idx_person,'Income'] = int(income_total)
            
        
        # If person is below 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                    
            
            # if the person (married or couple and below 25 years) does not have any children:       
            else:
                
                universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_extra_care
                
                income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                df_looking_after.at[idx_person,'Income'] = int(income_total)
            
     
    # If person is older or equal to 25:       
    else:
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
        
            # if the person (single and > 25 years) does not have any children:       
            else:
                
                
                universal_credit_income = universal_credit_standard_single_25plus + universal_credit_extra_care
                
                income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                df_looking_after.at[idx_person,'Income'] = int(income_total)
        
        # If person is above 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + universal_credit_children_second + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + (2* universal_credit_children_second) + universal_credit_extra_care
                
                    income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                    df_looking_after.at[idx_person,'Income'] = int(income_total)
            
            
            # if the person (couple/married and > 25 years) does not have any children:       
            else:
                
                
                universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_extra_care
                
                income_total = 52 * carer_allowance + 12 * universal_credit_income
                
                df_looking_after.at[idx_person,'Income'] = int(income_total)
                
    df_looking_after.at[idx_person,'Income'] = np.random.normal(int(df_looking_after.at[idx_person,'Income']),int(df_looking_after.at[idx_person,'Income'])*0.1, 1)


In [ ]:
df_looking_after['Income'].min()

In [ ]:
df_looking_after['Income'].max()

In [ ]:
df_looking_after.hist(column='Income', bins=50);

## other

In [ ]:
# Universal Credit
##You may be able to get Universal Credit if you’re on a low income or need help with your living costs. You could be working (including self-employed or part time) or be out of work.
#To claim you must:
##live in the UK
##be aged 18 or over (there are some exceptions if you’re 16 to 17)
##be under State Pension age
##have £16,000 or less in money, savings and investments
## https://www.gov.uk/universal-credit

universal_credit_standard_single_under25 = 265.31 #(£ extra Monthly standard allowance)
universal_credit_standard_single_25plus = 334.91 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_under25 = 416.45 #(£ extra Monthly standard allowance)
universal_credit_standard_couple_25plus = 525.72 #(£ extra Monthly standard allowance)

universal_credit_children_first = 290.0 #(£ extra Monthly)
universal_credit_children_second = 244.58 #(£ extra Monthly)


df_Other = df_persons_NE_inactive_5categories.loc[df_persons_NE_inactive_5categories['Occupation'] == 'Other']

df_Other['Income'] = 0


for idx_person, person in df_Other.iterrows():
    
    # If person is younger than 25:
    if person['Age'] < 25 :
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + universal_credit_children_second
                
                    income_total =  12 *universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_under25 + universal_credit_children_first + (2* universal_credit_children_second) 
                
                    income_total =  12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                    
            # if the person (single and below 25 years) does not have any children:       
            else:
                
                universal_credit_income = universal_credit_standard_single_under25 
                
                income_total = 12 * universal_credit_income
                
                df_Other.at[idx_person,'Income'] = int(income_total)
            
        
        # If person is below 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + universal_credit_children_second 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_under25/2 + universal_credit_children_first + (2* universal_credit_children_second) 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                    
            
            # if the person (married or couple and below 25 years) does not have any children:       
            else:
                
                universal_credit_income = universal_credit_standard_couple_under25/2 
                
                income_total = 12 * universal_credit_income
                
                df_Other.at[idx_person,'Income'] = int(income_total)
            
     
    # If person is older or equal to 25:       
    else:
        
        if person['Marital_status']  == 'Single':
        
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + universal_credit_children_second 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_single_25plus + universal_credit_children_first + (2* universal_credit_children_second) 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
        
            # if the person (single and > 25 years) does not have any children:       
            else:
                
                
                universal_credit_income = universal_credit_standard_single_25plus 
                
                income_total = 12 * universal_credit_income
                
                df_Other.at[idx_person,'Income'] = int(income_total)
        
        # If person is above 25 and is married or couple:   
        else:
            
            if person['Total_Children_in_household'] > 0:
            
                if person['Total_Children_in_household'] == 1:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
                
                
                if person['Total_Children_in_household'] == 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + universal_credit_children_second 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
            
                if person['Total_Children_in_household'] > 2:
                
                    
                    universal_credit_income = universal_credit_standard_couple_25plus/2 + universal_credit_children_first + (2* universal_credit_children_second) 
                
                    income_total = 12 * universal_credit_income
                
                    df_Other.at[idx_person,'Income'] = int(income_total)
            
            
            # if the person (couple/married and > 25 years) does not have any children:       
            else:
                
                
                universal_credit_income = universal_credit_standard_couple_25plus/2 
                
                income_total = 12 * universal_credit_income
                
                df_Other.at[idx_person,'Income'] = int(income_total)
    df_Other.at[idx_person,'Income'] = np.random.normal(int(df_Other.at[idx_person,'Income']),int(df_Other.at[idx_person,'Income'])*0.1, 1)


In [ ]:
df_Other['Income'].min()

In [ ]:
df_Other['Income'].max()

In [ ]:
df_Other.hist(column='Income', bins=50);

## Student

In [ ]:
#National Living Wage and National Minimum Wage: government response to the Low Pay Commission’s Autumn 2021 recommendations
##https://www.gov.uk/government/publications/national-living-wage-and-national-minimum-wage-government-response-to-the-low-pay-commissions-autumn-2021-recommendations

#Details
# £8.91 for workers aged 23 and over (the National Living Wage)
# £8.36 for 21 to 22-year-olds
# £6.56 for 18 to 20-year-olds
# £4.62 for 16 to 17-year-olds
# £4.30 for apprentices aged under 19 or in the first year of their apprenticeship


#Min and max wage per hours (rounded values)
## Min income refers to te national minimum wage per age.
## Maximum income is an estimate value chosen after several iterations
income_age_16_17 = [4, 6]
income_age_18_20 = [6, 11]
income_age_21_22 = [8, 16]
income_age_23_plus = [9,21]

# min and max hours worked per week:
weekly_working_hours = [5,20]


df_student = df_persons_NE_inactive_5categories.loc[df_persons_NE_inactive_5categories['Occupation'] == 'Student']

df_student['Income'] = 0


for idx_person, person in df_student.iterrows():
    
    
    if (person['Age'] >= 16  and  person['Age'] <= 17):
        
        df_student.at[idx_person,'Income'] = random.randrange(int(income_age_16_17[0]),int(income_age_16_17[1]), 1) * random.randrange(int(weekly_working_hours[0]),int(weekly_working_hours[1]), 1) * 52

    elif (person['Age'] >= 18  and  person['Age'] <= 20):
        
        df_student.at[idx_person,'Income'] = random.randrange(int(income_age_18_20[0]),int(income_age_18_20[1]), 1) * random.randrange(int(weekly_working_hours[0]),int(weekly_working_hours[1]), 1) * 52

    elif (person['Age'] >= 21  and  person['Age'] <= 22):
        
        df_student.at[idx_person,'Income'] = random.randrange(int(income_age_21_22[0]),int(income_age_21_22[1]), 1) * random.randrange(int(weekly_working_hours[0]),int(weekly_working_hours[1]), 1) * 52

    elif (person['Age'] >= 23):
        
        df_student.at[idx_person,'Income'] = random.randrange(int(income_age_23_plus[0]),int(income_age_23_plus[1]), 1) * random.randrange(int(weekly_working_hours[0]),int(weekly_working_hours[1]), 1) * 52



In [ ]:
df_student['Income'].min()

In [ ]:
df_student['Income'].max()

In [ ]:
df_student.hist(column='Income', bins=50);

In [ ]:
# Concatenate all dataframes updated with the income values:
df_persons_NE_inactive_5categories_income = (pd.concat([df_retired__updated_all, df_sick, df_looking_after, df_Other, df_student]))


In [ ]:
# Save dataframe
df_persons_NE_inactive_5categories_income_export_20220306_correct = df_persons_NE_inactive_5categories_income
df_persons_NE_inactive_5categories_income_export_20220306_correct.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_persons_NE_inactive_5categories_income_correct.csv', encoding='utf-8', header=True)


In [ ]:
df_persons_NE_inactive_5categories_income.hist(column='Income', bins=20);

In [ ]:
## there are a lot of retired people (age > 65)